Traitement des données pour vérifier s'il en manque



In [ ]:
!pip install housing

Importation des librairies


In [ ]:
import housing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tarfile
import os
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt




Extraction du/des fichiers


In [ ]:
file_path_general = 'general_data.csv'
file_path_employee ='employee_survey_data.csv'
file_path_manager = 'manager_survey_data.csv'
file_path_in = 'in_time.csv'
file_path_out = 'out_time.csv'



directory= "/content/Data/"

# Load the dataset
general_data = pd.read_csv(directory + file_path_general)
employee_data = pd.read_csv(directory + file_path_employee)
managers_data = pd.read_csv(directory + file_path_manager)
in_data = pd.read_csv(directory + file_path_in)
out_data = pd.read_csv(directory + file_path_out)


in_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
out_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

#general_data.info()

#general_data.describe()


general_data.head()



      Age Attrition     BusinessTravel              Department  \
0      51        No      Travel_Rarely                   Sales   
1      31       Yes  Travel_Frequently  Research & Development   
2      32        No  Travel_Frequently  Research & Development   
3      38        No         Non-Travel  Research & Development   
4      32        No      Travel_Rarely  Research & Development   
...   ...       ...                ...                     ...   
4405   42        No      Travel_Rarely  Research & Development   
4406   29        No      Travel_Rarely  Research & Development   
4407   25        No      Travel_Rarely  Research & Development   
4408   42        No      Travel_Rarely                   Sales   
4409   40        No      Travel_Rarely  Research & Development   

      DistanceFromHome  Education EducationField  EmployeeCount  EmployeeID  \
0                    6          2  Life Sciences              1           1   
1                   10          1  Life Sciences 

In [ ]:
employee_data.head()
managers_data.head()
in_data.head()
out_data.head()

,count
BusinessTravel,
Travel_Rarely,3129
Travel_Frequently,831
Non-Travel,450


Conversion In et Out


Fusion des dataframe par EmployeeID


In [ ]:
# Fusionner tous les DataFrames sur EmployeeID
merged_data = general_data.merge(employee_data, on='EmployeeID', how='outer') \
    .merge(managers_data, on='EmployeeID', how='outer') \
    .merge(in_data, on='EmployeeID', how='outer') \
    .merge(out_data, on='EmployeeID', how='outer')


merged_data.head()

In [ ]:


null_value = merged_data.isnull().sum()
null_value_manager = managers_data.isnull().sum()
null_value_employee = employee_data.isnull().sum()
null_value_in = in_data.isnull().sum()
null_value_out = out_data.isnull().sum()

print(null_value)


Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64


Remplissage des valeurs numériques manquantes par la valeure médianne


In [ ]:
import missingno as msno

# Visualize missing data
msno.matrix(merged_data)
plt.show()

# Impute missing values
# For numerical columns, use median imputation
numerical_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
merged_data[numerical_columns] = merged_data[numerical_columns].fillna(merged_data[numerical_columns].median())


In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

le=LabelEncoder()
ordinal=OrdinalEncoder()
#suppression des ligne avec valeurs manquante ? a check
general_data = general_data.dropna()

#on utilise l'encoder pour n'avoir que des valeurs numériques dans nos données

categorical_columns = ["Attrition", "BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "Over18"]

# for col in column_to_encode:
#     general_data[col]=le.fit_transform(general_data[col])
general_data['Attrition'] = le.fit_transform(general_data['Attrition'])
general_data[categorical_columns] = ordinal.fit_transform(general_data[categorical_columns])


general_data

NameError: name 'general_data' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Sélectionner les features pertinentes
features = [col for col in general_data.columns if col != 'Attrition' and col != 'EmployeeID']

X = general_data[features]
y = general_data['Attrition']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Régression logistique
log_reg = LogisticRegression(random_state=42, max_iter=5000)
log_reg.fit(X_train, y_train)

# Prédire et évaluer
y_pred_log = log_reg.predict(X_test)
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

# Importance des variables pour la régression logistique
importance = pd.DataFrame({'feature': X.columns, 'importance': abs(log_reg.coef_[0])})
importance = importance.sort_values('importance', ascending=False)
print("Top 10 features (Logistic Regression):")
print(importance.head(10))




In [ ]:

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Prédire et évaluer
y_pred_rf = rf.predict(X_test)
print("\nAccuracy (Random Forest):", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Importance des variables pour Random Forest
importance_rf = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_})
importance_rf = importance_rf.sort_values('importance', ascending=False)
print("\nTop 10 features (Random Forest):")
print(importance_rf.head(10))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculer les métriques pour les deux modèles
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
log_reg_scores = [
    accuracy_score(y_test, y_pred_log),
    precision_score(y_test, y_pred_log, average='weighted'),
    recall_score(y_test, y_pred_log, average='weighted'),
    f1_score(y_test, y_pred_log, average='weighted')
]
rf_scores = [
    accuracy_score(y_test, y_pred_rf),
    precision_score(y_test, y_pred_rf, average='weighted'),
    recall_score(y_test, y_pred_rf, average='weighted'),
    f1_score(y_test, y_pred_rf, average='weighted')
]

# Créer le graphique
x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, log_reg_scores, width, label='Logistic Regression')
rects2 = ax.bar(x + width/2, rf_scores, width, label='Random Forest')

ax.set_ylabel('Scores')
ax.set_title('Comparison of Logistic Regression and Random Forest')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

# Ajouter les valeurs sur les barres
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()